In [1]:
import lang2vec.lang2vec as l2v
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from pyglottolog import Glottolog
from scipy import spatial

In [83]:
# languages = ["nld", "eng", "ekk", "fin", "deu", "ell", "heb", "ita", "nor", "rus", "spa", "tur"]
languages = ["spa", "por", "cmn", "jpn"]
features =  ["syntax_wals", "syntax_sswl", "syntax_ethnologue"]

In [84]:
#URIEL data
df_uriel = pd.DataFrame.from_dict(l2v.get_features(languages, features))
df_uriel = df_uriel.replace("--", np.NaN).dropna(how='all')

imputer = KNNImputer(n_neighbors=5).fit(df_uriel).transform(df_uriel)
# imputer[imputer >= 0.5] = 1
# imputer[imputer < 0.5] = 0

df_uriel = pd.DataFrame(imputer, columns=df_uriel.columns)

nunique_uriel = df_uriel.nunique(axis=1)
df_uriel = df_uriel.drop(nunique_uriel[nunique_uriel == 1].index)

In [85]:
#Glottolog data
glottolog = Glottolog('../data/glottolog-4.6', cache=True)

glottolog_dict = {}
for l in languages:
    print(l)
    languoid = glottolog.languoid(l)
    families = {family:1 for family in [str(f.id) for f in languoid.ancestors]}
    glottolog_dict[l] = families
    
df_glottolog = pd.DataFrame.from_dict(glottolog_dict).fillna(0)

nunique = df_glottolog.nunique(axis=1)
df_glottolog = df_glottolog.drop(nunique[nunique == 1].index)

spa
por
cmn
jpn


In [86]:
#concat
df = pd.concat([df_uriel, df_glottolog])
df.transpose().to_csv("../data/outputs/linguistic_vectors.csv")

#compute cosine similarity
ls_dict = {}
for l1 in languages:
    ls_dict[l1] = {}
    for l2 in languages:
        ls = spatial.distance.cosine(df[l1], df[l2])
        ls_dict[l1][l2] = ls

df_ls = pd.DataFrame.from_dict(ls_dict).applymap(lambda x: 1 - x)
df_ls.to_csv("../data/outputs/linguistic_similarities.csv")

In [87]:
df_ls

,spa,por,cmn,jpn
spa,1.000000,0.921327,0.483046,0.135289
por,0.921327,1.000000,0.489188,0.156002
cmn,0.483046,0.489188,1.000000,0.504280
jpn,0.135289,0.156002,0.504280,1.000000


portuguese-spanish: 0.94 -> 0.95
chinese-japanese: 0.62 -> 0.5
chinese-portuguese: 0.59 -> 
chinese-spanish: 0.59
japanese-portuguese: 0.33
japanese-spanish:0.34